In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - AutoGluon

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_autogluon.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_autogluon.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>                                                                                               <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_autogluon.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10

## Overview

This notebook demonstrates finetuning a PyTorch based [Autogluon model for tabular data](https://auto.gluon.ai/stable/tutorials/tabular/index.html) on CPU and deploying it on Vertex AI for online prediction.

### Objective

In this tutorial, you learn how to:

- Finetune a PyTorch AutoGluon tabular model.
- Upload the model to [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model on [Endpoint](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for tabular data.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Training
- Vertex AI Model Registry
- Vertex AI Online Prediction

### Dataset

You can find the details for the [example dataset here](https://auto.gluon.ai/stable/tutorials/tabular/tabular-quick-start.html#example-data).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [2]:
# Install the packages.
! pip3 install --upgrade google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.43.0
    Uninstalling google-cloud-aiplatform-1.43.0:
      Successfully uninstalled google-cloud-aiplatform-1.43.0


### Colab only

In [1]:
# Automatically restart kernel after installs so that your environment can access the new packages.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying fine tuned model to Vertex AI endpoint.


#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
PROJECT_ID = "cmpt756-final-project-417723"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [4]:
from google.colab import auth
auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

In [5]:
# The service account for deploying fine tuned model.
# The service account looks like:
# '<account_name>@<project>.iam.gserviceaccount.com'
SERVICE_ACCOUNT = "service@cmpt756-final-project-417723.iam.gserviceaccount.com"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [14]:
BUCKET_URI = f"gs://cmpt756finalproject-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [15]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://cmpt756finalproject-cmpt756-final-project-417723-unique/...


### Import libraries

In [8]:
import os
from datetime import datetime

from google.cloud import aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [9]:
staging_bucket = os.path.join(BUCKET_URI, "autogluon_staging")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=staging_bucket)

### Define constants

In [10]:
# The pre-built training docker image.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-autogluon-train:20240124_0927_RC00"
# The pre-built serving docker image.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-autogluon-serve:20240124_0938_RC00"
# Serving port.
PORT = 8501

### Define common functions

This section defines functions for:

- Converting a Cloud Storage path such as `gs://bucket-name` to GCSFuse path format such as `/gcsfuse/bucket-name`.
- Deploy the trained model to Vertex AI Endpoint for prediction.

In [11]:
def gcs_fuse_path(path: str) -> str:
    """Try to convert path to gcsfuse path if it starts with gs:// else do not modify it."""
    path = path.strip()
    if path.startswith("gs://"):
        return "/gcs/" + path[5:]
    return path


def deploy_model(model_path):
    """Deploy the model to Vertex AI Endpoint for prediction."""
    model_name = "autogluon"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "model_path": model_path,
    }
    # Since the model_id is a GCS path, use artifact_uri to pass it
    # to the serving docker.
    artifact_uri = model_path
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[PORT],
        serving_container_predict_route="/predict",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-highmem-16",
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint

## Finetune with AutoGluon

Create and run the training job with the model-garden PyTorch AutoGluon training docker using the Vertex AI SDK. The training uses one CPU and runs for around 3 mins once the training job begins.

In [12]:
# Set up training docker arguments.

TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
JOB_NAME = "pytorch_autogluon" + TIMESTAMP

finetuning_workdir = os.path.join(BUCKET_URI, JOB_NAME)
train_data_path = (
    "https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/train.csv"
)
# The column id to predict.
label = "signature"

# We are using the
docker_args_list = [
    "--train_data_path",
    train_data_path,
    "--label",
    label,
    "--model_save_path",
    f"{gcs_fuse_path(finetuning_workdir)}",
]
print(docker_args_list)

['--train_data_path', 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/train.csv', '--label', 'signature', '--model_save_path', '/gcs/cmpt756finalproject-cmpt756-final-project-417723-unique/pytorch_autogluon20240320_003434']


In [16]:
# Create and run the training job.
# Click on the generated link in the output under "View backing custom job:" to see your run in the Cloud Console.
container_uri = TRAIN_DOCKER_URI
job = aiplatform.CustomContainerTrainingJob(
    display_name=JOB_NAME,
    container_uri=container_uri,
)
model = job.run(
    args=docker_args_list,
    base_output_dir=f"{finetuning_workdir}",
    replica_count=1,
    machine_type="n1-highmem-16",
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://cmpt756finalproject-cmpt756-final-project-417723-unique/pytorch_autogluon20240320_003434 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/9018429597787619328?project=678472902750
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/678472902750/locations/us-central1/trainingPipelines/9018429597787619328 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/82746977363689472?project=678472902750
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/678472902750/locations/us-central1/trainingPipelines/9018429597787619328 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob

## Run online prediction

Run online prediction with the trained model.

Upload the trained model and deploy it to an endpoint for prediction. This step takes around 20 minutes to finish.

In [17]:
model, endpoint = deploy_model(model_path=finetuning_workdir)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/678472902750/locations/us-central1/endpoints/8177122951351500800/operations/3732508858636042240
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/678472902750/locations/us-central1/endpoints/8177122951351500800
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/678472902750/locations/us-central1/endpoints/8177122951351500800')
INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/678472902750/locations/us-central1/models/1531858291515195392/operations/180857602500984832
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/678472902750/locations/us-central1/models/1531858291515195392@1
INFO:google.cloud.aiplatform.models:To use this Model in an

Send the prediction request for the query data. The expected `signature` label for this example query data is `-2`. You can also send comma separated multiple queries.

In [32]:
import time
instances = [
    {
        "Unnamed: 0": 70746,
        "chern_simons": 0.0905302166938781,
        "cusp_volume": 12.226321765565215,
        "hyperbolic_adjoint_torsion_degree": 0,
        "hyperbolic_torsion_degree": 10,
        "injectivity_radius": 0.5077560544013977,
        "longitudinal_translation": 10.685555458068848,
        "meridinal_translation_imag": 1.1441915035247805,
        "meridinal_translation_real": -0.5191566348075867,
        "short_geodesic_imag_part": -2.7606005668640137,
        "short_geodesic_real_part": 1.0155121088027954,
        "Symmetry_0": 0.0,
        "Symmetry_D3": 0.0,
        "Symmetry_D4": 0.0,
        "Symmetry_D6": 0.0,
        "Symmetry_D8": 0.0,
        "Symmetry_Z/2 + Z/2": 1.0,
        "volume": 11.393224716186523,
    },
]

start = time.time()

predictions = endpoint.predict(instances=instances).predictions

print("Prediction:", predictions, "Latency:", (time.time() - start))

Prediction: [-2.0] Latency: 0.3788630962371826


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete endpoint resource.
endpoint.delete(force=True)

# Delete model resource.
model.delete()

# Delete Cloud Storage objects that were created.
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI